In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import timeit

from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, precision_score, recall_score, accuracy_score, auc
from hmeasure import h_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, precision_recall_fscore_support,roc_auc_score, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

import optuna

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.layers import LeakyReLU
from keras.wrappers.scikit_learn import KerasClassifier
from keras import initializers
from keras.models import Sequential
from keras.optimizers import Adam

# Read the data using csv
path_train = r'C:\Users\.......\credit_risk_dataset_cleanedtraining.csv'
df_train = pd.read_csv(path_train, encoding = "ISO-8859-1")

path_test = r'C:\Users\......\credit_risk_dataset_cleanedtesting.csv'
df_test = pd.read_csv(path_test, encoding = "ISO-8859-1")

# Prin 5 forst lines of df
df_train.head()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,person_home_ownership,loan_intent,loan_grade,cb_person_default_on_file,loan_status
0,23,120000,1.0,25600,12.69,0.21,3,3,1,1,0,0
1,21,42500,3.0,25000,12.73,0.59,3,3,1,2,1,1
2,26,48000,2.0,25000,9.64,0.52,4,3,1,1,0,1
3,25,42360,1.0,25000,16.35,0.59,3,3,1,4,1,1
4,23,50000,1.0,25000,7.90,0.50,3,3,0,0,0,1


In [35]:
#pip install hmeasure

In [2]:
X_test, y_test = df_test.iloc[:,:-1], df_test.iloc[:,-1]

X_train, y_train = df_train.iloc[:,:-1], df_train.iloc[:,-1]

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2)

In [3]:
class tuning_methods:
    
    def __init__(self, X_train, X_val, y_train,  y_val, X_test, y_test):
        self.X_train = X_train
        self.X_val = X_val
        self.X_test = X_test
        self.y_test = y_test
        self.y_train = y_train
        self.y_val = y_val
        
    # Define the Keras model
    def create_model(self, units_1, alpha_1, dropout_1, l2_kern_1, l2_bias_1,units_2, alpha_2, dropout_2, l2_kern_2, l2_bias_2, lr, batch_size):        # 
        model = Sequential()
        model.add(Dense(units_1, 
                input_shape=(X_train.shape[1],), 
                activation = LeakyReLU(alpha=alpha_1),
                kernel_regularizer = l2(l2_kern_1),
                bias_regularizer = l2(l2_bias_1),
                kernel_initializer = initializers.RandomNormal(mean=0.0, stddev=0.05)
        ))
        model.add(Dropout(dropout_1))
    
        model.add(Dense(units_2,
                    activation =LeakyReLU(alpha=alpha_2),
                    kernel_regularizer = l2(l2_kern_2),
                    bias_regularizer = l2(l2_bias_2),
                    kernel_initializer = initializers.RandomNormal(mean=0.0, stddev=0.05)
        ))
        model.add(Dropout(dropout_2))
    
        model.add(Dense(1, activation='sigmoid'))
    
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer,
                  loss='binary_crossentropy', 
                  metrics=['AUC'])
        return model

    
    # Define the Optuna study
    def objective(self, trial):
        
        # Define the pipeline
        pipeline = Pipeline([
                ('scaler', StandardScaler()),
                ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))
                ])
        # Define hyperparameters range values for tuning
        units_1 = trial.suggest_int('units_1', 2, 40)
        units_2 = trial.suggest_int('units_2', 2, 40)
        alpha_1 = trial.suggest_float('alpha_1', 0.01, 0.1)
        alpha_2 = trial.suggest_float('alpha_2', 0.01, 0.1)
        dropout_1 = trial.suggest_float("dropout_1", 0.2, 0.5)    
        dropout_2 = trial.suggest_float("dropout_2", 0.2, 0.5)    
        l2_kern_1 = trial.suggest_float("l2_kern_1",  1e-5, 1e-2, log=True)
        l2_kern_2 = trial.suggest_float("l2_kern_2",  1e-5, 1e-2, log=True)
        l2_bias_1 = trial.suggest_float("l2_bias_1",  1e-5, 1e-2, log=True)
        l2_bias_2 = trial.suggest_float("l2_bias_2",  1e-5, 1e-2, log=True)
        lr = trial.suggest_float("lr", 5e-5, 1e-2, log=True)
        batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256, 512, 1024])
        epochs = 200
        validation_data = (self.X_val, self.y_val)
        early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        
        # Set parameters into pipeline
        pipeline.set_params(classifier__units_1=units_1,
                        classifier__alpha_1=alpha_1,
                        classifier__l2_kern_1=l2_kern_1,
                        classifier__l2_bias_1=l2_bias_1,
                        classifier__dropout_1=dropout_1,
                        classifier__units_2=units_2,
                        classifier__alpha_2=alpha_2,
                        classifier__l2_kern_2=l2_kern_2,
                        classifier__l2_bias_2=l2_bias_2,
                        classifier__dropout_2=dropout_2,
                        classifier__lr=lr,
                        classifier__epochs = epochs,
                        classifier__batch_size = batch_size,
                        classifier__validation_data = validation_data,
                        classifier__callbacks=[early_stopping])
        
        # Apply cross validation
        kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=2023)
        scores = cross_val_score(pipeline, self.X_train, self.y_train, cv=kfold, scoring="roc_auc", error_score='raise')
    
        if trial.should_prune():
                raise optuna.TrialPruned()

        return np.mean(scores)

    def Tuning(self, n_trials=100, sampler = optuna.samplers.TPESampler()):
        
        start = timeit.default_timer()

        study = optuna.create_study(direction="maximize",             
                            sampler=sampler, 
                            pruner= optuna.pruners.MedianPruner()
                           )
        study.optimize(self.objective, n_trials=n_trials)  # to be converged it needs atleast 200 trials 

        print("Number of finished trials: {}".format(len(study.trials)))

        print("Best trial:")
        trial = study.best_trial

        print("  Value: {}".format(trial.value))

        stop = timeit.default_timer()

        time_opt = stop - start
        params_TPE = []

        for key, value in trial.params.items():
            params_TPE.append([key,value])
            print("    {}: {}".format(key, value))
            
        params_TPE.append(["time",round(time_opt/60,2)])

        return params_TPE
    
    def Best_MLP_model(self, params):
        
        # Define hyperpameters
        units_1 = params[0][1]
        alpha_1 = np.round(params[2][1],3)
        dropout_1 = np.round(params[4][1],2)
        lr = np.round(params[10][1],5)
        batchs = params[11][1]
        kernel_regularizer_1=np.round(params[6][1],5)
        bias_regularizer_1=np.round(params[8][1],5)
        units_2 = params[1][1]
        alpha_2 = np.round(params[3][1],3)
        dropout_2 = np.round(params[5][1],2)
        kernel_regularizer_2=np.round(params[7][1],5)
        bias_regularizer_2=np.round(params[9][1],5)

        # Define the pipeline
        pipeline = Pipeline([
                    ('scaler', StandardScaler()),
                    ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))
                ])

        # Set hyperparameters
        epochs = 200
        validation_data = (self.X_val, self.y_val)
        early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        pipeline.set_params(classifier__units_1=units_1,
                            classifier__alpha_1 =alpha_1,
                            classifier__l2_kern_1=kernel_regularizer_1,
                            classifier__l2_bias_1=bias_regularizer_1,
                            classifier__dropout_1=dropout_1,
                            classifier__units_2=units_2,
                            classifier__alpha_2=alpha_2,
                            classifier__l2_kern_2=kernel_regularizer_2,
                            classifier__l2_bias_2=bias_regularizer_2,
                            classifier__dropout_2=dropout_2,
                            classifier__lr=lr,
                            classifier__epochs = epochs,
                            classifier__batch_size = batchs,
                            classifier__validation_data = validation_data,
                            classifier__callbacks=[early_stopping])
        
        # Use cross validation to monitor the deferrence between training and testing data as an evidence of overfitting
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2023)
        scores_train = cross_val_score(pipeline, self.X_train, self.y_train, cv=kfold, scoring="roc_auc", error_score='raise')
        mean_score_train = np.mean(scores_train)

        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2023)
        scores_test = cross_val_score(pipeline, self.X_test, self.y_test, cv=kfold, scoring="roc_auc", error_score='raise')
        mean_score_test = np.mean(scores_test)

        print(f"Train mean AUC score is {mean_score_train} while test mean AUC score is {mean_score_test}.")
      
        # Fit the pipeline to the training data
        history = pipeline.fit(self.X_train, self.y_train)

        #score = pipeline.evaluate(X_test, y_test, verbose=0)  

        y_pred_prob_tpe = pipeline.predict(self.X_test)

        y_pred_tpe = y_pred_prob_tpe.round()

        print(classification_report(self.y_test, y_pred_tpe, target_names = ['No','Yes']))

        acc = accuracy_score(self.y_test, y_pred_tpe)
        pre = precision_score(self.y_test, y_pred_tpe)
        rec = recall_score(self.y_test, y_pred_tpe)
        f1 = f1_score(self.y_test, y_pred_tpe)
        auc = roc_auc_score(self.y_test, y_pred_tpe)
        
        y_test_array = np.array(self.y_test)
        y_pred_tpe_array = np.array(y_pred_tpe)
        H_measure = h_score(y_test_array, y_pred_tpe_array)

        opt = [acc, pre, rec, f1, auc, H_measure]

        scores = pd.DataFrame(opt, index =['Accuracy', 'Precision', 'Recall', 'F1-score', 'AUC', "H-measure"], 
                              columns =['Tree-structured Parzen Estimator'])
        
        return scores 

## Take scores for every tuning algorithm

In [4]:
tun = tuning_methods(X_train, X_val, y_train,  y_val, X_test, y_test)
trials = 100

# Tree-structured Parzen Estimator
params_tpe = tun.Tuning(n_trials=trials, sampler = optuna.samplers.TPESampler())
tpe_scores = tun.Best_MLP_model(params_tpe)

[I 2023-03-02 12:24:01,186] A new study created in memory with name: no-name-4cb85434-67b9-40ac-805b-846d2dfd2dd7
C:\Users\User\AppData\Local\Temp\ipykernel_5968\4154129606.py:46: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 2ms/step


[I 2023-03-02 12:26:38,196] Trial 0 finished with value: 0.8592600484395626 and parameters: {'units_1': 6, 'units_2': 16, 'alpha_1': 0.08652392937806692, 'alpha_2': 0.043522788259195005, 'dropout_1': 0.4915772369800131, 'dropout_2': 0.4348032840990832, 'l2_kern_1': 0.0002963528728076074, 'l2_kern_2': 0.0005813178393895946, 'l2_bias_1': 0.0005282867499125279, 'l2_bias_2': 5.630690263356766e-05, 'lr': 0.002361051901665448, 'batch_size': 16}. Best is trial 0 with value: 0.8592600484395626.
C:\Users\User\AppData\Local\Temp\ipykernel_5968\4154129606.py:46: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 2ms/step


[I 2023-03-02 12:28:11,527] Trial 1 finished with value: 0.8750955208227996 and parameters: {'units_1': 14, 'units_2': 18, 'alpha_1': 0.08062014558237353, 'alpha_2': 0.07898875430356962, 'dropout_1': 0.330518284533719, 'dropout_2': 0.24091753980271113, 'l2_kern_1': 0.005877607204468142, 'l2_kern_2': 0.00017107390765937048, 'l2_bias_1': 0.0006159040105564795, 'l2_bias_2': 0.0008930704140011522, 'lr': 0.0020616143691081132, 'batch_size': 32}. Best is trial 1 with value: 0.8750955208227996.
C:\Users\User\AppData\Local\Temp\ipykernel_5968\4154129606.py:46: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


...

195/195 [==============================] - 0s 1ms/step


[I 2023-03-02 14:08:13,291] Trial 99 finished with value: 0.8819421013319083 and parameters: {'units_1': 34, 'units_2': 23, 'alpha_1': 0.051127381184261425, 'alpha_2': 0.016979673971436997, 'dropout_1': 0.4062321578471198, 'dropout_2': 0.21412158016520033, 'l2_kern_1': 9.619843445398418e-05, 'l2_kern_2': 0.00016756307542769742, 'l2_bias_1': 4.350422661287113e-05, 'l2_bias_2': 0.0003385245311863412, 'lr': 0.0011348888718008825, 'batch_size': 32}. Best is trial 95 with value: 0.8841611678688541.


Number of finished trials: 100
Best trial:
  Value: 0.8841611678688541
    units_1: 37
    units_2: 25
    alpha_1: 0.05210625251603535
    alpha_2: 0.02195344092214047
    dropout_1: 0.4072956196463625
    dropout_2: 0.25389495324898204
    l2_kern_1: 9.28915293064181e-05
    l2_kern_2: 0.00033272706769038536
    l2_bias_1: 1.786797740931158e-05
    l2_bias_2: 0.0004040644096473244
    lr: 0.0016828629000088273
    batch_size: 32


C:\Users\User\AppData\Local\Temp\ipykernel_5968\4154129606.py:140: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))


37/37 [==============================] - 0s 2ms/step
Train mean AUC score is 0.8839195673607371 while test mean AUC score is 0.8958723319517059.
183/183 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

          No       0.91      0.97      0.94      4648
         Yes       0.83      0.62      0.71      1186

    accuracy                           0.90      5834
   macro avg       0.87      0.79      0.82      5834
weighted avg       0.89      0.90      0.89      5834



In [6]:
# Random Sampler
params_rs = tun.Tuning(n_trials=trials, sampler = optuna.samplers.RandomSampler())
rs_scores = tun.Best_MLP_model(params_rs)
rs_scores = rs_scores.rename(columns={'Tree-structured Parzen Estimator': 'Random Sampler'})


[I 2023-03-02 03:28:21,612] A new study created in memory with name: no-name-711d97fb-40dd-4671-9bb7-eb85dc4855f4
C:\Users\User\AppData\Local\Temp\ipykernel_9660\4154129606.py:46: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 1ms/step


[I 2023-03-02 03:29:05,789] Trial 0 finished with value: 0.8743729991466039 and parameters: {'units_1': 20, 'units_2': 19, 'alpha_1': 0.0508767997767669, 'alpha_2': 0.02135056522662614, 'dropout_1': 0.3231322761875696, 'dropout_2': 0.4603393790639899, 'l2_kern_1': 0.007161915036401348, 'l2_kern_2': 0.0011940219496923764, 'l2_bias_1': 1.850113141009702e-05, 'l2_bias_2': 0.00808836146072827, 'lr': 0.0014636122779712187, 'batch_size': 64}. Best is trial 0 with value: 0.8743729991466039.
C:\Users\User\AppData\Local\Temp\ipykernel_9660\4154129606.py:46: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 2ms/step


[I 2023-03-02 03:30:04,980] Trial 1 finished with value: 0.866019269389315 and parameters: {'units_1': 17, 'units_2': 20, 'alpha_1': 0.018251306565743, 'alpha_2': 0.032861305776351636, 'dropout_1': 0.34534870247054483, 'dropout_2': 0.2190440589151384, 'l2_kern_1': 0.0005241337984031805, 'l2_kern_2': 0.007394191330339641, 'l2_bias_1': 4.159859096428988e-05, 'l2_bias_2': 0.001990372060250817, 'lr': 0.002734602232484103, 'batch_size': 64}. Best is trial 0 with value: 0.8743729991466039.
C:\Users\User\AppData\Local\Temp\ipykernel_9660\4154129606.py:46: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


...

195/195 [==============================] - 0s 1ms/step


[I 2023-03-02 05:33:38,626] Trial 99 finished with value: 0.8614392896471759 and parameters: {'units_1': 22, 'units_2': 29, 'alpha_1': 0.046233863255195295, 'alpha_2': 0.030221429455132648, 'dropout_1': 0.3141576104281736, 'dropout_2': 0.318249033662377, 'l2_kern_1': 1.151772315631499e-05, 'l2_kern_2': 4.178051897268625e-05, 'l2_bias_1': 0.00015073286553271477, 'l2_bias_2': 7.09552732499777e-05, 'lr': 0.0003565907276654163, 'batch_size': 512}. Best is trial 5 with value: 0.8903699507139367.


Number of finished trials: 100
Best trial:
  Value: 0.8903699507139367
    units_1: 37
    units_2: 23
    alpha_1: 0.03349519606693909
    alpha_2: 0.011575049992672148
    dropout_1: 0.2114537613928619
    dropout_2: 0.45429747486484096
    l2_kern_1: 1.2345640902292396e-05
    l2_kern_2: 0.0005339431188815503
    l2_bias_1: 0.00041779401566032886
    l2_bias_2: 0.0031130716173870935
    lr: 0.004086479715137101
    batch_size: 512


C:\Users\User\AppData\Local\Temp\ipykernel_9660\4154129606.py:140: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))


37/37 [==============================] - 0s 2ms/step
Train mean AUC score is 0.8882221421564302 while test mean AUC score is 0.8881831300232902.
183/183 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

          No       0.90      0.99      0.94      4648
         Yes       0.91      0.55      0.69      1186

    accuracy                           0.90      5834
   macro avg       0.90      0.77      0.81      5834
weighted avg       0.90      0.90      0.89      5834



In [5]:
# Genetic Algorithm
params_ga = tun.Tuning(n_trials=trials, sampler = optuna.samplers.NSGAIISampler())
ga_scores = tun.Best_MLP_model(params_ga)
ga_scores = ga_scores.rename(columns={'Tree-structured Parzen Estimator': 'Genetic Algorithm'})


[I 2023-03-02 14:13:49,317] A new study created in memory with name: no-name-648e3f0e-ed7c-4c55-8dc4-6609b6cf4535
C:\Users\User\AppData\Local\Temp\ipykernel_5968\4154129606.py:46: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 1ms/step


[I 2023-03-02 14:14:12,976] Trial 0 finished with value: 0.8647099322727357 and parameters: {'units_1': 25, 'units_2': 35, 'alpha_1': 0.03475615683698265, 'alpha_2': 0.09548130811993064, 'dropout_1': 0.28160612309881816, 'dropout_2': 0.28140127387018987, 'l2_kern_1': 8.651534041136919e-05, 'l2_kern_2': 0.007737546535067332, 'l2_bias_1': 0.0004885165137497015, 'l2_bias_2': 0.002298001801129686, 'lr': 0.0016902379815089633, 'batch_size': 1024}. Best is trial 0 with value: 0.8647099322727357.
C:\Users\User\AppData\Local\Temp\ipykernel_5968\4154129606.py:46: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 1ms/step


[I 2023-03-02 14:16:07,442] Trial 1 finished with value: 0.8521933990929157 and parameters: {'units_1': 14, 'units_2': 21, 'alpha_1': 0.030867899045047005, 'alpha_2': 0.08885139300038289, 'dropout_1': 0.4783843310779814, 'dropout_2': 0.3853305826100967, 'l2_kern_1': 0.0004931338370444792, 'l2_kern_2': 0.0013867108457293123, 'l2_bias_1': 0.003682087864417631, 'l2_bias_2': 4.178964413389416e-05, 'lr': 0.002658776839363301, 'batch_size': 16}. Best is trial 0 with value: 0.8647099322727357.
C:\Users\User\AppData\Local\Temp\ipykernel_5968\4154129606.py:46: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


...

195/195 [==============================] - 0s 2ms/step


[I 2023-03-02 17:06:05,464] Trial 99 finished with value: 0.8499168024242111 and parameters: {'units_1': 8, 'units_2': 38, 'alpha_1': 0.06068222443400352, 'alpha_2': 0.035218502021446496, 'dropout_1': 0.34553970492888536, 'dropout_2': 0.21792658546567475, 'l2_kern_1': 0.003074059485027563, 'l2_kern_2': 0.000814239078124287, 'l2_bias_1': 0.0004910323377268938, 'l2_bias_2': 0.00017139036160871717, 'lr': 0.0007553966274492467, 'batch_size': 64}. Best is trial 2 with value: 0.883048790570847.


Number of finished trials: 100
Best trial:
  Value: 0.883048790570847
    units_1: 34
    units_2: 21
    alpha_1: 0.04003928359586926
    alpha_2: 0.06210711632772143
    dropout_1: 0.2938399442882509
    dropout_2: 0.21500315917306256
    l2_kern_1: 0.00014436199771479758
    l2_kern_2: 3.0111156117759884e-05
    l2_bias_1: 6.698400814244221e-05
    l2_bias_2: 0.005658282165694526
    lr: 0.006154152102597497
    batch_size: 32


C:\Users\User\AppData\Local\Temp\ipykernel_5968\4154129606.py:140: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))


37/37 [==============================] - 0s 2ms/step
Train mean AUC score is 0.8829733888916103 while test mean AUC score is 0.8959644124381354.
183/183 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

          No       0.90      0.99      0.94      4648
         Yes       0.93      0.57      0.71      1186

    accuracy                           0.90      5834
   macro avg       0.92      0.78      0.83      5834
weighted avg       0.91      0.90      0.90      5834



In [6]:
# Quasi Monte Carlo
params_mc = tun.Tuning(n_trials=trials, sampler = optuna.samplers.QMCSampler())
mc_scores = tun.Best_MLP_model(params_mc)
mc_scores = mc_scores.rename(columns={'Tree-structured Parzen Estimator': 'Quasi Monte Carlo'})


C:\Users\User\AppData\Local\Temp\ipykernel_5968\3195711531.py:2: ExperimentalWarning: QMCSampler is experimental (supported from v3.0.0). The interface can change in the future.
  params_mc = tun.Tuning(n_trials=trials, sampler = optuna.samplers.QMCSampler())
[I 2023-03-02 17:11:05,789] A new study created in memory with name: no-name-c691b527-be03-4d12-910f-d8154b71adef
C:\Users\User\AppData\Local\Temp\ipykernel_5968\4154129606.py:46: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 1ms/step


[I 2023-03-02 17:11:27,346] Trial 0 finished with value: 0.8405245406825963 and parameters: {'units_1': 2, 'units_2': 13, 'alpha_1': 0.05160302813336891, 'alpha_2': 0.05829862543311441, 'dropout_1': 0.346649434556459, 'dropout_2': 0.3937460814027184, 'l2_kern_1': 9.418968402909818e-05, 'l2_kern_2': 0.00011879251702846256, 'l2_bias_1': 0.0003144340824319381, 'l2_bias_2': 0.00555316021412017, 'lr': 0.002502431063176007, 'batch_size': 128}. Best is trial 0 with value: 0.8405245406825963.
C:\Users\User\AppData\Local\Temp\ipykernel_5968\4154129606.py:46: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))
[W 2023-03-02 17:11:27,412] The parameter 'l2_kern_1' in trial#1 is sampled independently by using `RandomSampler` instead of `QMCSampler` (optimization performance may be degrad

195/195 [==============================] - 0s 2ms/step


[I 2023-03-02 17:12:17,112] Trial 1 finished with value: 0.8150136660677808 and parameters: {'units_1': 2, 'units_2': 2, 'alpha_1': 0.01, 'alpha_2': 0.01, 'dropout_1': 0.2, 'dropout_2': 0.2, 'l2_kern_1': 0.00044363333973084846, 'l2_kern_2': 0.0010042992890250243, 'l2_bias_1': 0.0005535335165235047, 'l2_bias_2': 0.003392759601926083, 'lr': 5.000000000000004e-05, 'batch_size': 1024}. Best is trial 0 with value: 0.8405245406825963.
C:\Users\User\AppData\Local\Temp\ipykernel_5968\4154129606.py:46: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))
[W 2023-03-02 17:12:17,116] The parameter 'batch_size' in trial#2 is sampled independently by using `RandomSampler` instead of `QMCSampler` (optimization performance may be degraded). `QMCSampler` does not support dynamic search space 

...

195/195 [==============================] - 0s 2ms/step


[I 2023-03-02 19:02:52,071] Trial 99 finished with value: 0.8502719989861132 and parameters: {'units_1': 32, 'units_2': 16, 'alpha_1': 0.023359375, 'alpha_2': 0.019140625, 'dropout_1': 0.27734375, 'dropout_2': 0.31015625, 'l2_kern_1': 2.788126665413133e-05, 'l2_kern_2': 0.00011341944035027563, 'l2_bias_1': 0.000333762469429204, 'l2_bias_2': 0.0028902639100224507, 'lr': 0.00029646420788443013, 'batch_size': 1024}. Best is trial 86 with value: 0.8850448085499383.


Number of finished trials: 100
Best trial:
  Value: 0.8850448085499383
    units_1: 40
    units_2: 40
    alpha_1: 0.05851562500000001
    alpha_2: 0.020546875
    dropout_1: 0.21640625000000002
    dropout_2: 0.24921875000000002
    l2_kern_1: 0.0018768842935762192
    l2_kern_2: 0.00046138396827332176
    l2_bias_1: 0.0008816830667755708
    l2_bias_2: 1.8105582430271218e-05
    lr: 0.0008696990313610563
    batch_size: 32


C:\Users\User\AppData\Local\Temp\ipykernel_5968\4154129606.py:140: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))


37/37 [==============================] - 0s 2ms/step
Train mean AUC score is 0.8863892928885626 while test mean AUC score is 0.8943558418945395.
183/183 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

          No       0.91      0.98      0.94      4648
         Yes       0.89      0.62      0.73      1186

    accuracy                           0.91      5834
   macro avg       0.90      0.80      0.84      5834
weighted avg       0.91      0.91      0.90      5834



## Optimal parameters comparison

In [15]:
params_all = pd.DataFrame(zip([item[1] for item in params_tpe], [item[1] for item in params_rs], 
                              [item[1] for item in params_ga], [item[1] for item in params_mc]),
                          index =['NEURONS LAYER 1 ','NEURONS LAYER 2', "AlPHA_l","AlPHA_2", 'DROPOUT LAYER 1 ','DROPOUT LAYER 2',
                                  "KERNEL REGULARIZER_1","KERNEL REGULARIZER_2","BIAS REGURALIZER_1","BIAS REGURALIZER_2",
                                  'LEARNING RATE',"BATCH SIZE"] ,
                          columns = ['TPE','RS','GA', 'MC'])

params_all

,TPE,RS,GA,MC
NEURONS LAYER 1,37.000000,37.000000,34.000000,40.000000
NEURONS LAYER 2,25.000000,23.000000,21.000000,40.000000
AlPHA_l,0.052106,0.033495,0.040039,0.058516
AlPHA_2,0.021953,0.011575,0.062107,0.020547
DROPOUT LAYER 1,0.407296,0.211454,0.293840,0.216406
DROPOUT LAYER 2,0.253895,0.454297,0.215003,0.249219
KERNEL REGULARIZER_1,0.000093,0.000012,0.000144,0.001877
KERNEL REGULARIZER_2,0.000333,0.000534,0.000030,0.000461
BIAS REGURALIZER_1,0.000018,0.000418,0.000067,0.000882
BIAS REGURALIZER_2,0.000404,0.003113,0.005658,0.000018


## Metrics Summary

In [16]:
all_scores = pd.concat([tpe_scores, rs_scores, ga_scores, mc_scores], axis =1)
all_scores

,Tree-structured Parzen Estimator,Tree-structured Parzen Estimator,Genetic Algorithm,Quasi Monte Carlo
Accuracy,0.895955,0.900583,0.904354,0.907439
Precision,0.826381,0.915068,0.931319,0.887290
Recall,0.618044,0.563238,0.571669,0.623946
F1-score,0.707188,0.697286,0.708464,0.732673
AUC,0.792456,0.774949,0.780456,0.801861
H-measure,0.487065,0.466846,0.480633,0.518255


## Save results

In [17]:
# Save the clean dataset

all_scores.to_csv(r'C:\Users\........\Credit_risk_dataset_2_Layer_scores.csv', index=False)

params_all.to_csv(r'C:\Users\.........\Credit_risk_dataset_2_Layer_stracture.csv', index=False)
